# Ceneo Scraper

# Załadowanie bibliotek

In [14]:
import requests
from bs4 import BeautifulSoup

# Wysłanie zapytania do serwera


In [6]:
product_id= '138331381'
url=f'https://www.ceneo.pl/{product_id}#tab=reviews_scroll'
response = requests.get(url)
response.status_code

200

## Pobranie opinii z kodu HTML strony

In [9]:
page_dom = BeautifulSoup(response.text, 'html.parser')
print(type(page_dom))
opinions= page_dom.select('div.js_product-review')
print(type(opinions))
opinion=opinions.pop(0)
print(type(opinion))

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>


In [15]:
print(opinion)

<div class="user-post user-post__card js_product-review" data-entry-id="18497226">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/2.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
m...r</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2024-03-16 13:07:37">2 tygodnie temu, </time>
<time datetime="2024-03-08 18:30:25">po 2 tygodniach</time> użytkowania            </span>
</span>
<div class="user-post__text">Ładniejszy niż na fotce. J

## Ekstrakcja składowej pojedynczych opinii
| Składowa | Selektor | Zmienna |
|----------|--------|---------|
| id_opinii |div.js_product-review['data-entry-id']  | opinion_id | 
| autor |span.post_author-name | author | 
| rekomendacje |span.user-post_author-recomendation | recommendations | 
| gwiazdki |span.user-post_score-count |  stars | 
| treść |div.user-post_text | content | 
| lista zalet |div.review-feature_item-positives ~ div.review-feature_item | pros| 
| lista wad |div.review-feature_item-negatives ~ div.review-feature_item | cons |
| dla ilu przydatna |button.vote-yes > span | helpful |
| dla ilu nieprzydatna |button.vote-no > span | unhelpful |
| data wystawienia | user-post_published > time:nth-child(1)["datatime"] | publish_date |
| data zakupu |time:nth-child(2)["datatime"]| purchase_date |


In [ ]:
opinion_id=opinion['data-entry-id']